In [82]:
import pandas as pd
import os
import time

In [83]:
data_folder='./Data/'

In [84]:
file_channels = open(data_folder + "channel_ids.txt", "r")
file_pagerank_all= open(data_folder + "pagerank_all.txt", "r")
file_pagerank_2018= open(data_folder + "pagerank_2018_2019.txt", "r")

data_channels = file_channels.read()
data_pagerank_all = file_pagerank_all.read()
data_pagerank_2018 = file_pagerank_2018.read()

list_channels = data_channels.split("\n")
list_channels = list_channels[:-1] #discrading the last entry as it contains no information (residual from the split)
list_pagerank_all = data_pagerank_all.split("\n")
list_pagerank_all= list_pagerank_all[3:-2]#discrading first 3 values and the last one as they are not channel
#and the one before the last one as it doesn't contain anything (residual from the split)
list_pagerank_2018 = data_pagerank_2018.split("\n")
list_pagerank_2018= list_pagerank_2018[3:-2]#discrading first 3 values and the last one as they are not channel
#and the one before the last one as it doesn't contain anything (residual from the split)

In [85]:
print(len(list_channels), len(list_pagerank_all), len(list_pagerank_2018)) #all good the lengths match

136470 136470 136470


In [86]:
#creating a dataframe
zipped = list(zip(list_channels, list_pagerank_all, list_pagerank_2018))
df_cluster=pd.DataFrame(zipped, columns=['channel_id', 'pagerank', 'pagerank_2018'])

In [99]:
#notice that the way our data was stored was so that if we stack them next to each other, each line had a channel_id and
#the channel's associated pagerank cluster coefficient for all videos and the associated pagerank cluster coefficient
#for the time window 2018-2019
#converting our values into floats as they were under the string format
df_cluster['pagerank']=df_cluster['pagerank'].astype('float')
df_cluster['pagerank_2018']=df_cluster['pagerank_2018'].astype('float')
df_cluster

,channel_id,pagerank,pagerank_2018
0,UC-lHJZR3Gqxm24_Vd_AJ5Yw,176.551392,195.492874
1,UCbCmjCuTUZos6Inko4u57UQ,0.895367,1.312383
2,UCpEhnqL0y41EpW2TvWAHD7Q,6.800401,6.447253
3,UC295-Dw_tDNtZXFeAPAW6Aw,28.799509,37.452736
4,UCJ5v_MCY6GNUBTO8-D3XoAg,32.658581,32.937046
...,...,...,...
136465,UCuM-9AajUOwKw6ipOzu2DRQ,0.005047,0.009096
136466,UCtW9jp5TH0YrgYpwiRf9t-Q,0.014430,0.004309
136467,UCTsxFTIUs8vFDzGccDm6i7Q,0.186187,0.260013
136468,UC1HOArgRCMGPjlcmkThERwA,0.008244,0.000000


In [103]:
#counting the nomber of Nan values
nb_Nan=len(df_cluster[df_cluster.isnull().any(axis=1)])
print("there are", nb_Nan, "Nan values which corresponds to", nb_Nan*100/len(df_cluster), "% of the dataframe")

there are 0 Nan values which corresponds to 0.0 % of the dataframe


In [108]:
df_cluster[["pagerank", "pagerank_2018"]].describe()

,pagerank,pagerank_2018
count,136470.000000,136470.000000
mean,0.482503,0.434065
std,2.118560,2.053294
min,0.000000,0.000000
25%,0.020596,0.004830
50%,0.079321,0.042644
75%,0.277383,0.217053
max,176.551392,195.492874


### Comparing these values with the ones of the extreme communities channels

In [109]:
path_sources_final='sources_final.csv'
df_sources_final=pd.read_csv(data_folder + path_sources_final) #maps channel_id to extreme grp corresponding
df_sources=df_sources_final[["Category", "Id"]]
df_sources.rename(columns = {'Id':'channel_id'}, inplace = True)

C:\Users\HOURS\AppData\Local\Temp\ipykernel_5984\100672720.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sources.rename(columns = {'Id':'channel_id'}, inplace = True)


In [110]:
df_sources_interest=df_sources[df_sources['Category'].isin(['Alt-lite', 'Alt-right', 'Intellectual Dark Web'])]

In [119]:
df_sources_interest['Category'].value_counts()

Alt-lite                 113
Intellectual Dark Web     89
Alt-right                 88
Name: Category, dtype: int64

In [111]:
#page rank coeff for all and only 2018-2019 for only the channels of interest
df_cluster_interest=df_cluster[df_cluster['channel_id'].isin(list_channels_interest)]

In [117]:
df_cluster_interest=df_cluster.merge(df_sources_interest, how='right', on='channel_id')
df_cluster_interest.head()

,channel_id,pagerank,pagerank_2018,Category
0,UCfDdlNLRVb1h3_7Xh-WhL3w,11.852910,22.547094,Alt-lite
1,UCso6oQ3qy7z61Fr8b0WNBMw,NaN,NaN,Alt-lite
2,UCplXdu0g94UynzYCmO2zOjw,0.502852,0.948694,Alt-lite
3,UCl3RCEtooHD5bhPCHJw3btA,1.433627,0.683660,Alt-lite
4,UCbukZErY9SjdGTHabMzY7qg,0.000000,0.000000,Alt-lite


In [113]:
#counting the nomber of Nan values
nb_Nan=len(df_cluster_interest[df_cluster_interest.isnull().any(axis=1)])
print("there are", nb_Nan, "Nan values which corresponds to", nb_Nan*100/len(df_cluster_interest), "% of the dataframe")

there are 134 Nan values which corresponds to 46.206896551724135 % of the dataframe


In [114]:
#checking if rows where pagerank is Nan are also rows where pagerank_2018 is Nan
(df_cluster_interest[df_cluster_interest['pagerank'].isnull()]).equals(df_cluster_interest[df_cluster_interest['pagerank_2018'].isnull()])

True

In [115]:
#we choose to discard these rows as they don't bring new information
#dropping NaN values
df_cluster_interest.dropna(inplace=True)
nb_Nan=len(df_cluster_interest[df_cluster_interest.isnull().any(axis=1)])
print("there are", nb_Nan, "Nan values which corresponds to", nb_Nan*100/len(df_cluster_interest), "% of the dataframe")

there are 0 Nan values which corresponds to 0.0 % of the dataframe


In [116]:
df_cluster_interest['Category'].value_counts()

Alt-lite                 79
Intellectual Dark Web    57
Alt-right                20
Name: Category, dtype: int64

**Comment:** We have lost some information, all the extreme communities channel weren't in our cluster analysis. That could be because we discarded edges where less 1% of the commentators commented on another channel. As it reduced the size of our graph by 90%, it makes sense that we discarded the channels that had to little influence.

In [120]:
#converting our values into floats as they were under the string format
df_cluster_interest['pagerank']=df_cluster_interest['pagerank'].astype('float')
df_cluster_interest['pagerank_2018']=df_cluster_interest['pagerank_2018'].astype('float')

In [121]:
df_temp=df_cluster_interest.drop(['channel_id'], axis=1)
df_cluster_interest.groupby(by='Category').mean()

,pagerank,pagerank_2018
Category,,
Alt-lite,4.854888,6.121196
Alt-right,1.105379,1.435813
Intellectual Dark Web,3.202381,4.015690
